# Camada Bronze — Dados Brutos

Nesta etapa inicial, é feita a configuração da estrutura lógica onde todo o pipeline de dados será organizado dentro do Databricks. Para isso, criamos um catálogo, que funciona como um contêiner principal e os schemas responsáveis por agrupar as tabelas de cada camada do modelo de dados.

O catálogo será utilizado como ambiente padrão do projeto e dentro dele serão criados os schemas referentes às camadas Bronze, Silver e Gold, seguindo as boas práticas de arquitetura Lakehouse.
Essa estrutura garante organização, governança e facilidade de acesso aos dados ao longo do pipeline.

In [0]:
%sql
DROP CATALOG IF EXISTS mvp CASCADE

In [0]:
%sql
CREATE CATALOG mvp

In [0]:
%sql
USE CATALOG mvp

In [0]:
%sql
DROP SCHEMA IF EXISTS bronze CASCADE

In [0]:
%sql
CREATE SCHEMA bronze

In [0]:
%sql
DROP SCHEMA IF EXISTS silver CASCADE

In [0]:
%sql
CREATE SCHEMA silver

In [0]:
%sql
DROP SCHEMA IF EXISTS gold CASCADE

In [0]:
%sql
CREATE SCHEMA gold

In [0]:
df = (
    spark.read.format("csv")
    .option("header", "true")
    .option("multiline", "true")
    .option("wholeFile", "true")           
    .option("escape", "\"")
    .option("quote", "\"")
    .option("delimiter", ",")
    .load("dbfs:/Workspace/mvp_natalia/titles.csv")
)

df.count()



In [0]:
spark.sql("DROP TABLE IF EXISTS mvp.bronze.titles")
df.write.format("delta").saveAsTable("mvp.bronze.titles")


Nesta etapa, o arquivo titles.csv foi recarregado manualmente no Databricks utilizando PySpark, com parâmetros específicos de leitura (como multiline, escape, quote e delimiter) para garantir a correta interpretação do arquivo CSV e evitar inconsistências na estrutura dos dados. Após a validação da carga (total de 3.030 registros), os dados foram persistidos em formato Delta na camada bronze, com a recriação da tabela mvp.bronze.titles. Essa abordagem permitiu eliminar erros de parsing e problemas de formatação, assegurando que a base bruta estivesse íntegra e pronta para as etapas de transformação subsequentes.

In [0]:
spark.table("mvp.bronze.titles").count()


In [0]:
%sql
SELECT * FROM mvp.bronze.titles
LIMIT 50


In [0]:
%sql
DESCRIBE TABLE mvp.bronze.titles

Os arquivos CSV foram carregados com sucesso para o schema correspondente à camada Bronze.
Essa etapa representa a ingestão inicial dos dados no Lakehouse, preservando seu formato bruto para garantir rastreabilidade e fidelidade à fonte original.

Com os dados agora armazenados na Bronze, eles estão prontos para serem explorados e transformados nas próximas etapas do pipeline, onde serão limpos, padronizados e estruturados para compor as camadas Silver e Gold.